# Setup

In [1]:
%%capture
!sudo add-apt-repository -y ppa:openjdk-r/ppa
!sudo apt-get purge openjdk-*
!sudo apt-get install openjdk-8-jdk
!sudo apt-get install xvfb xserver-xephyr vnc4server python-opengl ffmpeg

In [2]:
%%capture
!pip install --upgrade 'setuptools==57.5.0'
!pip install 'gym==0.19.0'


In [3]:
!pip install --upgrade wheel==0.38.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: wheel
    Found existing installation: wheel 0.40.0
    Uninstalling wheel-0.40.0:
      Successfully uninstalled wheel-0.40.0


In [4]:
#%%capture
!pip install --upgrade minerl
!pip install pyvirtualdisplay

!pip install scikit-learn
!pip install -U colabgymrender

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 MB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached gym-0.19.0.tar.gz (1.6 MB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.0 MB/s eta 0:00:00
  Using cached cloudpickle-1.6.0-py3-none-any.whl (23 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.8 MB/s eta 0:00:00
  Created wheel for minerl: filename=minerl-0.4.4-cp310-cp310-linux_x86_64.whl size=563653752 sha256=7b75bdaba9563a6297baf86baf10979abb1480f3bbe4fb6ca60dbe27ff204c79
  Stored in directory: /root/.

In [5]:
# download if needed
# # !apt update 
# !apt install -y python-numpy python-dev cmake zlib1g-dev libjpeg-dev xvfb ffmpeg xorg-dev python-opengl libboost-all-dev libsdl2-dev swig

# !pip3 install pyvirtualdisplay piglet gym torch torchvision
# # !pip3 install "gym[atari]"

In [6]:
!apt-get install xorg openbox
!apt-get install xvfb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  apport apport-symptoms aspell aspell-en bubblewrap desktop-file-utils
  dictionaries-common dmidecode docbook-xml dosfstools eject emacsen-common
  enchant-2 gdisk giblib1 gnome-icon-theme gnome-terminal gnome-terminal-data
  gstreamer1.0-gl gstreamer1.0-plugins-base gstreamer1.0-plugins-good
  gstreamer1.0-pulseaudio gstreamer1.0-x gvfs gvfs-common gvfs-daemons
  gvfs-libs hunspell-en-us keyboard-configuration libaa1 libaspell15
  libatasmart4 libblockdev-crypto2 libblockdev-fs2 libblockdev-loop2
  libblockdev-part-err2 libblockdev-part2 libblockdev-swap2 libblockdev-utils2
  libblockdev2 libcdparanoia0 libdv4 libenchant-2-2 libgck-1-0 libgcr-base-3-1
  libgpgme11 libgraphene-1.0-0 libgstreamer-gl1.0-0
  libgstreamer-plugins-good1.0-0 libharfbuzz-icu0 libhunspell-1.7-0 libhyphen0
  libid3tag0 libimlib2 libjavascriptcoregtk-4.0-18 libloc

# Import Libraries

In [7]:
import random
import numpy as np
import torch as th
from torch import nn
import gym
import minerl
from tqdm.notebook import tqdm
from colabgymrender.recorder import Recorder
from gym.wrappers import Monitor
from time import time
import torch

from pyvirtualdisplay import Display
from sklearn.cluster import KMeans
import logging
logging.disable(logging.ERROR) # reduce clutter, remove if something doesn't work to see the error logs.

/usr/local/lib/python3.10/dist-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


# Parameters

In [8]:
# Parameters:
TEST_EPISODES = 10  # number of episodes to test the agent for.
MAX_TEST_EPISODE_LEN = 18000  # 18k is the default for MineRLObtainDiamondVectorObf.

# Download the data

In [9]:
minerl.data.download(directory='data', environment='MineRLObtainDiamond-v0');

Download: https://minerl.s3.amazonaws.com/v4/MineRLObtainDiamond-v0.tar: 3650.0MB [00:58, 62.76MB/s]                                


# Train

In [10]:
!unzip '/content/trained_model.zip' #change to your trained model path

Archive:  /content/trained_model.zip
   creating: CompareRL/Minecraft_proj/agents/MineRL2020/train/
   creating: CompareRL/Minecraft_proj/agents/MineRL2020/train/MineRLObtainDiamondVectorObf-v0/
   creating: CompareRL/Minecraft_proj/agents/MineRL2020/train/MineRLObtainDiamondVectorObf-v0/n_clusters_120/
   creating: CompareRL/Minecraft_proj/agents/MineRL2020/train/MineRLObtainDiamondVectorObf-v0/n_clusters_120/random_state_1/
  inflating: CompareRL/Minecraft_proj/agents/MineRL2020/train/MineRLObtainDiamondVectorObf-v0/n_clusters_120/random_state_1/kmeans.joblib  
  inflating: CompareRL/Minecraft_proj/agents/MineRL2020/train/model_10.tm  


In [11]:
!cp -r 'CompareRL/Minecraft_proj/agents/MineRL2020/train/' 'train/' #change first path to your train path (keep train/ as it is)

In [12]:
from model import Model


In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
curr_model=Model()
curr_model.load_state_dict(torch.load("train/model_10.tm", map_location=device))
curr_model.to(device)

Model(
  (core): Core(
    (input_proc): InputProcessor(
      (conv_layers): FixupResNetCNN(
        (conv_layers): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (2): _FixupResidual(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          )
          (3): _FixupResidual(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          )
          (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (5): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (6): _FixupResidual(
            (con

In [ ]:
# stops the display
# display.stop()
# !rm -rf /content/logs
# !rm -rf /content/video

In [15]:
# restarts the display
display = Display(visible=0, size=(400, 300))
display.start();

In [16]:
env_D = gym.make('MineRLObtainDiamondVectorObf-v0')
env_D = Recorder(env_D, './video', fps=60)

In [20]:
seeds = range(10)

In [21]:
stats = {'runtime': [], 'reward': [],'reward_at':[]}

In [22]:
TEST_EPISODES=10
for episode in range(TEST_EPISODES):
  with torch.no_grad():
    rewards=[]
    reward_sum=0
    steps=1
    env_D.seed(seeds[episode])
    obs = env_D.reset()
    done = False
    start = time()
    state = curr_model.get_zero_state(1, device=device)
    s = torch.zeros((1,1,64), dtype=torch.float32, device=device)
    while not done:
      spatial = torch.tensor(obs["pov"], device=device, dtype=torch.float32).unsqueeze(0).unsqueeze(0).transpose(2,4)
      nonspatial = torch.cat([torch.tensor(obs["vector"], device=device, dtype=torch.float32), 
                              torch.ones((2,), device=device,dtype=torch.float32)], dim=0).unsqueeze(0).unsqueeze(0)
      s, state = curr_model.sample(spatial, nonspatial, s, state, torch.zeros((1,1,64),dtype=torch.float32,device=device))
        
      obs,reward,done,_ = env_D.step({"vector":s})
      steps+=1
      if steps%1000==0:
        print(f"step: {steps}")
      reward_sum += reward
      if reward > 0:
        rewards.append((steps,reward))
        print(reward_sum)
      if reward==256:
        print("Met target")
        break
      if MAX_TEST_EPISODE_LEN <= steps:
        print("Miss target 2")
        break
      elif done:
        print("Miss target 1")
        break
    
    stats['runtime'].append(time() - start)
    stats['reward'].append(reward_sum)
    stats['reward_at'].append(rewards)

step: 1000
step: 2000
step: 3000
step: 4000
step: 5000
step: 6000
step: 7000
step: 8000
step: 9000
step: 10000
step: 11000
step: 12000
step: 13000
step: 14000
step: 15000
step: 16000
step: 17000
Miss target 1
step: 1000
Miss target 1
1.0
step: 1000
step: 2000
step: 3000
3.0
7.0
11.0
19.0
step: 4000
step: 5000
35.0
step: 6000
step: 7000
step: 8000
step: 9000
step: 10000
step: 11000
step: 12000
step: 13000
step: 14000
step: 15000
step: 16000
step: 17000
Miss target 1
1.0
step: 1000
step: 2000
step: 3000
step: 4000
step: 5000
step: 6000
step: 7000
step: 8000
Miss target 1
1.0
Miss target 1
1.0
step: 1000
3.0
step: 2000
step: 3000
7.0
step: 4000
11.0
step: 5000
step: 6000
Miss target 1
step: 1000
step: 2000
step: 3000
step: 4000
step: 5000
Miss target 1
1.0
step: 1000
step: 2000
3.0
step: 3000
7.0
step: 4000
11.0
step: 5000
step: 6000
step: 7000
19.0
step: 8000
step: 9000
step: 10000
step: 11000
step: 12000
step: 13000
step: 14000
step: 15000
step: 16000
step: 17000
Miss target 1
step: 100

In [23]:
# saves the stats
import json
with open('stats_imi.json', 'w') as outfile:
    json.dump(stats, outfile)

In [24]:
!zip -r video video/

  adding: video/ (stored 0%)
  adding: video/1685543460.3749018.mp4 (deflated 1%)
  adding: video/1685544280.438004.mp4 (deflated 1%)
  adding: video/1685542198.8287334.mp4 (deflated 3%)
  adding: video/1685541278.7279367.mp4 (deflated 1%)
  adding: video/1685544355.5210183.mp4 (deflated 1%)
  adding: video/1685542326.9986815.mp4 (deflated 1%)
  adding: video/1685546526.869451.mp4 (deflated 1%)
  adding: video/1685544806.5045006.mp4 (deflated 1%)
  adding: video/1685546805.902302.mp4 (deflated 2%)
  adding: video/1685545330.9752715.mp4 (deflated 1%)
